# Microsoft SEAL : Programming with PyEva (EVA compiler)

In this example, I compute the following formula, in which $ x $ is encrypted by homomorphic encryption (HE) library, Microsoft SEAL.<br>
Here I use EVA compiler (and PyEva) to simplify programming. Please compare with [previous exercise](./02-seal-python-ckks-with-c-wrapper.ipynb) to see how it simplifies.

$ 3.14159265 x^3 + 0.5 $

for $ x=0.0, 1.1, 2.2, 3.3 $

Before starting, please install and set up software as follows.<br>
(Here I used **Ubuntu 20.04 LTS** in Microsoft Azure.)

```
# Install dependencies
sudo apt-get update
sudo apt-get install -y python3-pip
sudo -H pip3 install --upgrade pip
sudo apt install cmake libboost-all-dev libprotobuf-dev protobuf-compiler
# Install CLang
sudo apt install clang
sudo update-alternatives --install /usr/bin/cc cc /usr/bin/clang 100
sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/clang++ 100
# Install Microsoft SEAL version 3.6
git clone -b v3.6.4 https://github.com/microsoft/SEAL.git
cd SEAL
cmake -DSEAL_THROW_ON_TRANSPARENT_CIPHERTEXT=OFF .
make -j
sudo make install
cd ..
# Install EVA
git clone https://github.com/microsoft/EVA
cd EVA
git submodule update --init
cmake .
make -j
# Install Python packages
python3 -m pip install -e python/
pip3 install numpy
```

*back to [Readme](https://github.com/tsmatz/homomorphic-encryption-microsoft-seal/)*

In [1]:
#
# Define and compile program
#

# Define program
from eva import *
poly = EvaProgram("Polynomial", vec_size=1024)
with poly:
    x = Input("x")
    Output("y", 3.14159265*x**3 + 0.5)
poly.set_output_ranges(30)
poly.set_input_scales(30)
# Compile program with CKKS scheme
from eva.ckks import *
compiler = CKKSCompiler()
compiled_poly, params, signature = compiler.compile(poly)

In [2]:
#
# Generate key context
#   public context contains : public key, relin key, galois key
#   secret context contains : secret key
#
from eva.seal import *
public_ctx, secret_ctx = generate_keys(params)

In [3]:
#
# Create encryption for x (= 0.0, 1.1, 2.2, 3.3)
#
inputs = { "x": [0.0 for i in range(compiled_poly.vec_size)] }
inputs["x"][0] = 0.0
inputs["x"][1] = 1.1
inputs["x"][2] = 2.2
inputs["x"][3] = 3.3
encInputs = public_ctx.encrypt(inputs, signature)

In [4]:
#
# Execute computation
#
encOutputs = public_ctx.execute(compiled_poly, encInputs)

In [5]:
#
# Decrypt results
#
outputs = secret_ctx.decrypt(encOutputs, signature)
print("********** Result is **********")
for i in range(4):
    print(outputs["y"][i])

********** Result is **********
0.5000000000005966
4.681459288719608
33.951451702453895
113.39935418382298
